# Worksheet 4: Investigating Extremes

This final worksheet is a chance for you to put together all the techniques you have learnt in the last three worksheets.

Working with the same data-sets, write some code to undertake an analysis of temperature extremes over our South East Asia domain.

<div class="alert alert-block alert-warning">
<b>By the end of this worksheet you should be able to:</b><br> 
- Build a coding workflow to undertake extremes analysis. <br>
- Understand the different stages of post-processing.<br>
- Have an appreciation of data storage and file naming issues.
</div>

## Preamble

In [ ]:
# Code preamble - these libraries will be used in this worksheet.
# This code block needs to be re-run every time you restart this worksheet!
%matplotlib inline 
import os
import iris
import iris.coord_categorisation
import iris.quickplot as qplt
import iris.plot as iplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import dask
dask.config.set(get=dask.get)
import dask.array as da
from iris.analysis import Aggregator

# /data/users/ssadri/pyprecis/monthly
# Some helpful data locations
DATADIR = '/project/precis/worksheets/data'
APHRODIR = os.path.join(DATADIR, 'APHRODITE')
CRUDIR = os.path.join(DATADIR, 'CRU')
CLIMDIR = os.path.join(DATADIR, 'climatology')
MODELDIR = os.path.join(DATADIR, 'netcdf')
JOBIDS = ['cahpa', 'cahpb']
TIME_PERIODS = {'baseline':'1981_1983', 'future':'2021_2050'}

## Step 1: Process raw pp files

Remember to:
 * Select the right variable from the pp file
 * Remove the rim
 * Concatenate into a single netCDF file
 * Tidy cube metadata
 * Save the netCDF file so you can access it later

In [ ]:
'''
Step 1: Process the raw pp files
'''


## Step 2: Calculate the extreme indicie

Remember to:
  * Consider which observational data set to use
  * Calulate the indicie for both the model data you prepared in step 1, but also the observed data
  * Calculate the indicie for present and future model periods
  * Save your indicie data sets as a netCDF file with appropriate metadata

In [ ]:
'''
Step 2: Calculate the extreme indicie
'''


## Step 3: Plot some results

Remember to:
  * Think about plotting absolute values, differences, or scaled differences.  Which one communicates your message best?
  * Choose an appropriate colour scale

In [ ]:
'''
Step 3: Plot some results
'''


© Crown Copyright 2018, Met Office